In [3]:
import collections
import numpy as np
import numpy.linalg

In [4]:
def _estimate_fopdt(u, y, deadtime):
    
    b = y[1 + deadtime:] # this is y(n+1) assume the dead time is zero
    
    A = np.empty((len(b), 3))
    A[:, 0] = y[deadtime:-1] # this is y(n), assuming the dead time is zero
    A[:, 1] = u[:-(1 + deadtime)] # this is u(n) assuming the dead time iz zero
    A[:, 2] = 1.
    x, residuals, rank, s = np.linalg.lstsq(A, b) # fit  x1y(n)+ x2u(n) + x3 = y(n+1)
    #if including dead time , this will be x1 y(n+dt)+ x2u(n) +x3 = y(n+1+deadtime)
    return np.append(x, deadtime), residuals, rank, s

In [5]:
import pandas as pd

In [6]:
df1 = pd.read_excel(r"D:\SMOC simulations\first order model data.xls")

In [7]:
b1 = df1['POV_1']

In [8]:
b = b1[6:]

In [9]:
u1 = df1['MV_1']

In [10]:
u = u1[:-6]

In [11]:
A = np.empty((len(b),3))

In [12]:
A[:,0] = b

In [13]:
A[:,1] = u

In [14]:
A[:,2]= 1

In [15]:
x, residuals, rank ,s = np.linalg.lstsq(A,b)

In [16]:
x

array([  1.00000000e+00,  -9.44096331e-17,   6.34983606e-16])

In [17]:
rank

3

In [18]:
residuals

array([  2.25558038e-26])

In [19]:
s

array([ 916.10736317,  192.11720742,    1.14765509])

numpy.linalg.lstsq(a, b, rcond=-1)[source]
Return the least-squares solution to a linear matrix equation.

Solves the equation a x = b by computing a vector x that minimizes the Euclidean 2-norm || b - a x ||^2. The equation may be under-, well-, or over- determined (i.e., the number of linearly independent rows of a can be less than, equal to, or greater than its number of linearly independent columns). If a is square and of full rank, then x (but for round-off error) is the “exact” solution of the equation.

Parameters:	
a : (M, N) array_like
“Coefficient” matrix.
b : {(M,), (M, K)} array_like
Ordinate or “dependent variable” values. If b is two-dimensional, the least-squares solution is calculated for each of the K columns of b.
rcond : float, optional
Cut-off ratio for small singular values of a. Singular values are set to zero if they are smaller than rcond times the largest singular value of a.
Returns:	
x : {(N,), (N, K)} ndarray
Least-squares solution. If b is two-dimensional, the solutions are in the K columns of x.
residuals : {(), (1,), (K,)} ndarray
Sums of residuals; squared Euclidean 2-norm for each column in b - a*x. If the rank of a is < N or M <= N, this is an empty array. If b is 1-dimensional, this is a (1,) shape array. Otherwise the shape is (K,).
rank : int
Rank of matrix a.
s : (min(M, N),) ndarray
Singular values of a.
Raises:	
LinAlgError :
If computation does not converge.

In [20]:
class FOPDT(object):
    """Implements a discrete first order plus dead time model.
    
    The corresponding difference equation is::
    
        y[n + 1] = a * y[n] + b * u[n - deadtime] + c

    with::

        a = exp(dt / time_constant)
        b = (1 - a) * gain
        c = (1 - a) * steady_state
    
    ::

        u, y = np.loadtxt('testdata.dat', unpack=True)
        #Fit model to data
        model = FOPDT.fit(u, y, deadtime=range(0, 20), dt=.5)
    
        # Simulate Output
        uo, yo = model.output(u, y0=y[0:model.deadtime + 1])

    """
    def __init__(self, a, b, c, deadtime_shift, dt=1):
        self.a = a
        self.b = b
        self.c = c
        self.deadtime_shift = int(deadtime_shift)
        self.dt = dt

    def output(self, u, y0=None):
        """Calculates the response of the FOPDT model
        to the given input vector.
        
        :param u: The input vector.
        :param y0: The initial condition.
        
        """
        deadtime = self.deadtime_shift
        y = np.zeros_like(u)
        if y0 is not None:
            y[:deadtime + 1] = y0
        for i in range(len(y) - (1 + deadtime)):
            y[i + 1 + deadtime] = y[i + deadtime] * self.a + u[i] * self.b + self.c
            
        return u, y
  
    @staticmethod
    def fit(u, y, deadtime_shift, dt=1):
        """Fits a first order plus deadtime model to the data.
        
        :param u: The input vector.
        :param y: The output vector.
        :param dt: The sampling time.
        :param deadtime_shift: The dead time in integer multiples of the
            sampling time. If the deadtime is known beforehand it can
            be used directly. Otherwise a range of deadtimes
            should be specified. The deadtime giving the best fit
            is then used.

        :returns: A :class:`~ .FOPDT` instance.
        
        """
        assert len(u) == len(y)
        if isinstance(deadtime_shift, collections.Iterable):
            fits = [_estimate_fopdt(u, y, dtime) for dtime in deadtime_shift]
            # get fit with smallest residual error
            x, residuals, rank, s = min(fits, key=lambda f: f[1])
        else:
            x, residuals, rank, s = _estimate_fopdt(u, y, deadtime_shift)
            
        return FOPDT(*x, dt=dt)
    
    @property
    def time_constant(self):
        """The time constant of the process model."""
        return - self.dt / np.log(self.a)
    
    @property
    def gain(self):
        """The gain of the process model."""
        return self.b / (1. - self.a)
    
    @property
    def steady_state(self):
        """The steady state value of the process model."""
        return self.c / (1. - self.a)
        
    @property
    def deadtime(self):
        """The process deadtime."""
        return self.deadtime_shift * self.dt

In [21]:
df1

,Timestamp,MV_1,POV_1,POV_2
0,2014-01-01 00:00:00,40,19.999998,100.000000
1,2014-01-01 00:01:00,40,19.845085,99.833244
2,2014-01-01 00:02:00,40,20.100760,99.810684
3,2014-01-01 00:03:00,40,20.086212,99.685471
4,2014-01-01 00:04:00,45,19.968718,99.968391
5,2014-01-01 00:05:00,45,20.349396,99.788040
6,2014-01-01 00:06:00,45,19.944633,99.984329
7,2014-01-01 00:07:00,45,19.812309,100.182076
8,2014-01-01 00:08:00,45,19.730682,99.837532
9,2014-01-01 00:09:00,45,20.031174,99.932014


In [22]:
model_pov1 = FOPDT.fit(df1['MV_1'],df1['POV_1'],range(0,20))

In [23]:
model_pov2 = FOPDT.fit(df1['MV_1'],df1['POV_2'],range(0,20))

In [24]:
print(" Modle 1 d = " ,model_pov1.deadtime, "g = " ,model_pov1.gain, "tau = " ,model_pov1.time_constant)

 Modle 1 d =  5 g =  2.99009357707 tau =  9.75320114676


model 1 parameters are ( d = 5, g = 2.99 , time constant = 9.75320 ) where as aida parameters yield  5, 2.995, 9.9982 

In [25]:
print("Model 2 d = " ,model_pov2.deadtime, "g = " ,model_pov2.gain, "tau = " ,model_pov2.time_constant)

Model 2 d =  10 g =  0.964071104534 tau =  17.5116542226


### it is interesting to note that the transfer fucntion parameters are identified by linear least squares and not curve fitting 